In [4]:
import pandas as pd
import numpy as np
import os 


In [5]:
os.getcwd()
os.chdir('C:\\Users\\colli_000\\Documents\\titanic')
os.getcwd()

'C:\\Users\\colli_000\\Documents\\titanic'

In [6]:
os.listdir()

['gender_submission.csv', 'test.csv', 'train.csv']

In [7]:
train = pd.read_csv('./train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
## I suspect that Name, Ticket, and Cabin don't have much predictive value
## dropping these fields
train = train.drop(['Name','Ticket','Cabin'], axis=1)
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [9]:
## the Age and Embarked fields have Null values
train.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Sex            False
Age             True
SibSp          False
Parch          False
Fare           False
Embarked        True
dtype: bool

In [10]:
## will linear interolate the age
train = train.interpolate(method='linear', axis=0)

In [11]:
## still null values in the 'Embarked' field
train.isna().any()

PassengerId    False
Survived       False
Pclass         False
Sex            False
Age            False
SibSp          False
Parch          False
Fare           False
Embarked        True
dtype: bool

In [13]:
## for the embarked class I'll replace the Nulls with a new value N
train = train.fillna('N')
train.isna().any()

PassengerId    False
Survived       False
Pclass         False
Sex            False
Age            False
SibSp          False
Parch          False
Fare           False
Embarked       False
dtype: bool

In [14]:
train_X = train.drop(['Survived','PassengerId'], axis=1)
train_X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,22.5,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [15]:
train_y = train[['Survived']]
train_y

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


Nominal Variable (Categorical). Variable comprises a finite set of discrete values with no relationship between values.
Ordinal Variable. Variable comprises a finite set of discrete values with a ranked ordering between values.

In this data set, an example of an Ordinal Variable is Pclass - it is categorical, but there is an ordered relation between the classes. (Fortunately this class is already numerically encoded, so I don't have to do anything here)

Examples of Nominal Variables are 'Sex' and 'Embarked'. For these I will have to encode them. Since they are Nominal, it is more appropriate to One-Hot Encode them, than to encode them as if an ordinal relationship exists.

One-Hot Encoding
For categorical variables where no ordinal relationship exists, the integer encoding may not be enough, at best, or misleading to the model at worst.

Forcing an ordinal relationship via an ordinal encoding and allowing the model to assume a natural ordering between categories may result in poor performance or unexpected results (predictions halfway between categories).

One-Hot encoding - This is where the integer encoded variable is removed and one new binary variable is added for each unique integer value in the variable.

There seems to be two different options for One-Hot Encoding.
You can use OneHotEncoder from sklearn.preprocessing..


In [16]:
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder = OneHotEncoder(sparse=False)
oneHotEncoder.fit_transform(train_X[['Sex','Embarked']])


array([[0., 1., 0., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0.]])

.. but it looks like the pandas method 'get_dummies'does the same thing, but easier - it automatically names your new binary variables


In [17]:
train_X = pd.get_dummies(train_X)

In [18]:
train_X

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_N,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,1,0,0,0,1
887,1,19.0,0,0,30.0000,1,0,0,0,0,1
888,3,22.5,1,2,23.4500,1,0,0,0,0,1
889,1,26.0,0,0,30.0000,0,1,1,0,0,0


At this point I'm wondering what the appropriate way to normalise my features is.
Do I normalise the continuous features such as Age and Fare, and leave the other features as they are?
It seems to be a topic of debate, but most people seem to say you should treat them separately.

"It depends on your ML algorithms, some methods requires almost no efforts to normalize features or handle both continuous and discrete features, like tree based methods: c4.5, Cart, random Forrest, bagging or boosting. But most of parametric models (generalized linear models, neural network, SVM,etc) or methods using distance metrics (KNN, kernels, etc) will require careful work to achieve good results. Standard approaches including binary all features, 0 mean unit variance all continuous features, etc"

https://www.quora.com/How-do-I-deal-with-normalization-of-feature-vectors-when-they-contain-both-discrete-and-continuous-valued-features
https://www.quora.com/What-are-good-ways-to-handle-discrete-and-continuous-inputs-together

Now I want to use StandardScaler to scale the Age and Fare fields, and not the others. There seems to be no built in way to do this (except something called ColumnTransformer https://stackoverflow.com/questions/38420847/apply-standardscaler-to-parts-of-a-data-set). But I will do it the long way.

In [19]:
## extracting the continuous features I want to scale
from sklearn.preprocessing import StandardScaler
features_to_scale = train_X[['Age','Fare']]

## scaling them
scaled_features = StandardScaler().fit_transform(features_to_scale.values)

In [20]:
scaled_features

array([[-0.55604973, -0.50244517],
       [ 0.59548094,  0.78684529],
       [-0.26816707, -0.48885426],
       ...,
       [-0.5200644 , -0.17626324],
       [-0.26816707, -0.04438104],
       [ 0.16365693, -0.49237783]])

In [21]:
## and then reassigning the new scaled values back into my DataFrame
train_X[['Age','Fare']] = scaled_features

In [22]:
train_X

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_N,Embarked_Q,Embarked_S
0,3,-0.556050,1,0,-0.502445,0,1,0,0,0,1
1,1,0.595481,1,0,0.786845,1,0,1,0,0,0
2,3,-0.268167,0,0,-0.488854,1,0,0,0,0,1
3,1,0.379569,1,0,0.420730,1,0,0,0,0,1
4,3,0.379569,0,0,-0.486337,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,2,-0.196196,0,0,-0.386671,0,1,0,0,0,1
887,1,-0.771962,0,0,-0.044381,1,0,0,0,0,1
888,3,-0.520064,1,2,-0.176263,1,0,0,0,0,1
889,1,-0.268167,0,0,-0.044381,0,1,1,0,0,0


For many data scientists, a typical workflow consists of using Pandas to do exploratory data analysis before moving to scikit-learn for machine learning.

Second, it's important to remember is that Scikit-Learn exclusively works with array-like objects. It expects that X has shape (n_samples, n_features) and y to have shape (n_samples,)

It will check for these shapes when you use fit, so if your X, y don't abide by these rules, it will crash. Good news, X already has shape (5,2), but y will have shape (5, 1), which is different than (5,) and so your program might crash.

To be safe, I'd simply transform my X and y as numpy arrays from the start.

X = pd.DataFrame(np.ones((5, 2)))
y = pd.DataFrame(np.ones((5,)))

X = np.array(X)
y = np.array(y).squeeze()

In [23]:
# converting my pandas DataFrames into numpy arrays - the scikit's expected input
train_X = np.array(train_X)
train_y = np.array(train_y).squeeze()

In [24]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.33, random_state=42)

In [25]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)


SVC()

In [26]:
# making predictions on test set
y_pred = model.predict(X_test)

# calculating accuracy of predictions
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.8338983050847457

In [129]:
trainingData = pd.read_csv('./train.csv')
trainingData = trainingData.drop(['Name','Ticket','Cabin','PassengerId'], axis=1)
trainingData.isna().any()
trainingData = trainingData.dropna(axis = 0)
trainingData.isna().any()

Survived    False
Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
Embarked    False
dtype: bool

In [304]:
from sklearn.preprocessing import StandardScaler
import random
def preprocess_data(data, isTestData):
    data = data.drop(['Name','Ticket','Cabin','PassengerId'], axis=1)
    # replacing the NaNs in Age with an interpolated value
    data = data.interpolate(method='linear', axis=0)
    # replacing the NaNs in Embarked with a random letter
    data['Embarked'] = data['Embarked'].replace(np.nan,['S','C','Q'][random.randint(0,2)])

    data = pd.get_dummies(data)
    
    ## extracting the continuous features I want to scale
    features_to_scale = data[['Age','Fare']]
    ## scaling them
    scaled_features = StandardScaler().fit_transform(features_to_scale.values)
    ## and then reassigning the new scaled values back into my DataFrame
    data[['Age','Fare']] = scaled_features
    return data

def get_target_variable(data):
    train_X = data.drop(['Survived'], axis=1)
    train_y = data[['Survived']]
    
    # train test split - not going to split training data for final model training
    #from sklearn.model_selection import train_test_split
    #X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.33, random_state=42)
    
    return train_X,train_y

def convert_to_numpy(data):    
    # converting my pandas DataFrames into numpy arrays - the scikit's expected input
    # applying squeeze - if training set is passed in squeeze will have no effect,
    # if 1-dimensional target variable is passed in squeeze will change to 1-d numpy array
    data = np.array(data).squeeze()
    return data
        

def train_model(X_train, y_train):
    from sklearn.svm import SVC
    model = SVC()
    model.fit(X_train, y_train)
    return model

def make_predictions(X_test, model):
    y_pred = model.predict(X_test)
    return y_pred

In [309]:
def main():
    ## training portion
    trainingData = pd.read_csv('./train.csv')
    trainingData = preprocess_data(trainingData, False)
    train_X, train_y = get_target_variable(trainingData)
    train_X = convert_to_numpy(train_X)
    train_y = convert_to_numpy(train_y)
    fitted_model = train_model(train_X,train_y)
    
    ## making predictions on test data
    testData = pd.read_csv('./test.csv')
    output = testData[['PassengerId']]
    testData = preprocess_data(testData, True)
    testData = convert_to_numpy(testData)
    y_pred = make_predictions(testData, fitted_model)
    output['Survived'] = y_pred
    output.to_csv('my_submission.csv', index=False)
    print('output saved as my_submission.csv')
    return output



In [310]:
main()

output saved as my_submission.csv


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
